### Imports

In [5]:
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from pathlib import Path


### Load Datasets

In [6]:
data_path = Path.cwd()

# Pfad zur CSV-Datei das Später erstellt wird
train = pd.read_csv(data_path/'../csv/feature_data/train_engineered.csv')
test = pd.read_csv(data_path/'../csv/feature_data/test_engineered.csv')

### Modell

In [7]:
# Überprüfe, ob im Testdatensatz 'Survived' fehlt und setze es auf -1, um die Struktur beizubehalten
test['Survived'] = -1

# Bereinige die Daten, falls nötig (z.B. fehlende Werte)
# Für das Beispiel hier keine Bereinigung angenommen.

train =  train.drop(['Ticket'], axis=1)
test = test.drop(['Ticket'], axis=1)

# Feature und Label definieren
X_train = train.drop(['PassengerId', 'Survived','AverageSurvivalRate'], axis=1)
y_train = train['Survived']

# Modell vorbereiten
model = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=100, random_state=7))

# Cross-Validation
cv = KFold(n_splits=10, random_state=42, shuffle=True)
scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
print(f'Accuracy scores for the 10 folds: {scores}')
print(f'Average accuracy: {scores.mean()}')

# Modell mit dem gesamten Trainingsdatensatz trainieren
model.fit(X_train, y_train)

# Vorhersagen für Testdatensatz machen
X_test = test.drop(['PassengerId', 'Survived','AverageSurvivalRate'], axis=1)
predictions = model.predict(X_test)

# Vorhersagen in den Testdatensatz einfügen und in CSV exportieren
test['Survived'] = predictions
result = test[['PassengerId', 'Survived']]
result.to_csv(data_path/'../csv/submission_test.csv', index=False)

Accuracy scores for the 10 folds: [0.86666667 0.82022472 0.80898876 0.85393258 0.85393258 0.86516854
 0.76404494 0.80898876 0.79775281 0.88764045]
Average accuracy: 0.8327340823970036


Um das Modell zu optimieren, gibt es mehrere Ansätze, die du in Betracht ziehen kannst. Ein paar gängige Methoden umfassen die Auswahl von besseren Features, die Anwendung von Hyperparameter-Tuning und die Erprobung verschiedener Modelle. In diesem Beispiel konzentrieren wir uns auf das Hyperparameter-Tuning für den `RandomForestClassifier` mithilfe von `GridSearchCV` für eine systematische Durchsuchung eines Parameterbereichs.

```python
import numpy as np
from sklearn.model_selection import GridSearchCV

# Hyperparameter, die optimiert werden sollen
param_grid = {
    'randomforestclassifier__n_estimators': [100, 200, 300],
    'randomforestclassifier__max_depth': [None, 10, 20, 30],
    'randomforestclassifier__min_samples_split': [2, 5, 10],
    'randomforestclassifier__min_samples_leaf': [1, 2, 4]
}

# GridSearchCV vorbereiten
grid_search = GridSearchCV(model, param_grid, cv=cv, scoring='accuracy', verbose=1, n_jobs=-1)

# Suche ausführen
grid_search.fit(X_train, y_train)

# Beste Parameter und Genauigkeit anzeigen
print(f'Best parameters: {grid_search.best_params_}')
print(f'Best score: {grid_search.best_score_}')

# Vorhersagen mit dem optimierten Modell machen
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)

# Vorhersagen in den Testdatensatz einfügen und in CSV exportieren
test['Survived'] = predictions
result_optimized = test[['PassengerId', 'Survived']]
result_optimized.to_csv('submission_test_optimized.csv', index=False)
```

Ein paar Punkte zur Berücksichtigung:

- **Grid Search** kann sehr zeitaufwendig sein, insbesondere bei einer großen Anzahl von Hyperparametern und wenn du `n_jobs=-1` setzt, um alle Prozessoren zu verwenden. Überlege dir, ob du eine kleinere Auswahl an Werten für den ersten Durchlauf testen möchtest.
- **Hyperparameter-Bereiche**: Die hier gewählten Werte (`n_estimators`, `max_depth`, `min_samples_split`, `min_samples_leaf`) sind gängige Startpunkte, aber je nach deinem spezifischen Datensatz könnten andere Werte oder zusätzliche Hyperparameter interessant sein.
- **Modellauswahl**: RandomForest ist ein starker Baseline-Classifier, aber je nach den Eigenschaften deines Datensatzes könnten auch andere Modelle wie Gradient Boosting Machines (z.B. XGBoost, LightGBM) oder sogar einfache Modelle wie Logistische Regression besser performen.
- **Feature Engineering**: Es könnte hilfreich sein, zusätzliches Feature Engineering durchzuführen, um die Modellleistung weiter zu verbessern. Dies könnte das Erstellen neuer Features, die Auswahl spezifischer Features oder das Anwenden von Feature Transformationen beinhalten.

Durch das Anwenden dieser Optimierungstechniken solltest du in der Lage sein, die Leistung deines Modells weiter zu verbessern.

In [8]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import make_pipeline

# # Daten laden
# train = pd.read_csv("csv/train_engineered.csv")
# test = pd.read_csv("csv/test_engineered.csv")

# # Vorbereitung
# test['Survived'] = -1
# train = train.drop(['Ticket'], axis=1)
# test = test.drop(['Ticket'], axis=1)

# # Features und Labels definieren
# X_train = train.drop(['PassengerId', 'Survived', 'AverageSurvivalRate'], axis=1)
# y_train = train['Survived']

# # Modell vorbereiten mit einem geänderten Zufallszustand für Konsistenz
# model = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=7))

# # Hyperparameter für GridSearch
# param_grid = {
#     'randomforestclassifier__n_estimators': [100, 200, 300],
#     'randomforestclassifier__max_depth': [None, 10, 20, 30],
#     'randomforestclassifier__min_samples_split': [2, 5, 10],
#     'randomforestclassifier__min_samples_leaf': [1, 2, 4]
# }

# # GridSearchCV vorbereiten
# cv = KFold(n_splits=10, random_state=42, shuffle=True)
# grid_search = GridSearchCV(model, param_grid, cv=cv, scoring='accuracy', verbose=1, n_jobs=-1)

# # Suche ausführen
# grid_search.fit(X_train, y_train)

# # Beste Parameter und Genauigkeit anzeigen
# print(f'Best parameters: {grid_search.best_params_}')
# print(f'Best score: {grid_search.best_score_}')

# # Vorhersagen mit dem optimierten Modell machen
# X_test = test.drop(['PassengerId', 'Survived', 'AverageSurvivalRate'], axis=1)
# predictions = grid_search.best_estimator_.predict(X_test)

# # Vorhersagen in den Testdatensatz einfügen und in CSV exportieren
# test['Survived'] = predictions
# result_optimized = test[['PassengerId', 'Survived']]
# result_optimized.to_csv('submission_test_optimized.csv', index=False)


In [9]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import make_pipeline
# from sklearn.decomposition import PCA

# # Daten laden
# train = pd.read_csv("csv/train_engineered.csv")
# test = pd.read_csv("csv/test_engineered.csv")

# # Vorbereitung
# test['Survived'] = -1
# train = train.drop(['Ticket'], axis=1)
# test = test.drop(['Ticket'], axis=1)

# # Features und Labels definieren
# X_train = train.drop(['PassengerId', 'Survived', 'AverageSurvivalRate'], axis=1)
# y_train = train['Survived']

# # Modell Pipeline vorbereiten mit PCA
# model = make_pipeline(
#     StandardScaler(),
#     PCA(n_components=0.95),  # Behalte 95% der Varianz
#     RandomForestClassifier(random_state=7)
# )

# # Hyperparameter für GridSearch, jetzt mit PCA Komponenten
# param_grid = {
#     'pca__n_components': [0.80, 0.85, 0.90, 0.95],  # Unterschiedliche Varianzanteile, die beibehalten werden sollen
#     'randomforestclassifier__n_estimators': [100, 200, 300],
#     'randomforestclassifier__max_depth': [None, 10, 20, 30],
#     'randomforestclassifier__min_samples_split': [2, 5, 10],
#     'randomforestclassifier__min_samples_leaf': [1, 2, 4]
# }

# # GridSearchCV vorbereiten
# cv = KFold(n_splits=10, random_state=42, shuffle=True)
# grid_search = GridSearchCV(model, param_grid, cv=cv, scoring='accuracy', verbose=1, n_jobs=-1)

# # Suche ausführen
# grid_search.fit(X_train, y_train)

# # Beste Parameter und Genauigkeit anzeigen
# print(f'Best parameters: {grid_search.best_params_}')
# print(f'Best score: {grid_search.best_score_}')

# # Vorhersagen mit dem optimierten Modell machen
# X_test = test.drop(['PassengerId', 'Survived', 'AverageSurvivalRate'], axis=1)
# predictions = grid_search.best_estimator_.predict(X_test)

# # Vorhersagen in den Testdatensatz einfügen und in CSV exportieren
# test['Survived'] = predictions
# result_optimized = test[['PassengerId', 'Survived']]
# result_optimized.to_csv('submission_test_optimized.csv', index=False)
